In [1]:
PORT = 5000
env = 'ICL' # LOCAL | ICL

In [15]:
from pathlib import Path

if env == 'LOCAL':
    WORK_DIR = Path('.')
elif env == 'ICL':
    import os, sys
    WORK_DIR = Path('/vol/bitbucket/') / os.getenv("USER") / 'worker'
    # !pip install -r {WORK_DIR}/requirements.txt

    if not WORK_DIR in sys.path:
        sys.path.append(str(WORK_DIR))

os.environ['WORK_DIR'] = str(WORK_DIR)

['/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/vol/bitbucket/kza23/msc/lib/python3.10/site-packages', '/vol/bitbucket/kza23/worker']


In [16]:
import uvicorn
import nest_asyncio
from fastapi import FastAPI, WebSocket, BackgroundTasks
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
from src.constants import UI_URL, WORKER_URL
from src.models.requests import ProcessRequest, WorkUnit
from src.socket_service import SocketService
from src.work import process_work
from dummy_response import audit
import logging

nest_asyncio.apply()

app = FastAPI()

origins = [
    str(UI_URL),
    str(WORKER_URL),
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post("/process")
async def process(request: ProcessRequest, background_tasks: BackgroundTasks):
    SocketService.register_incoming_id(request.socket)
    background_tasks.add_task(process_work, request.work)
    return JSONResponse(content={"status": 200})

@app.websocket("/ws/{socket_id}")
async def websocket_endpoint(websocket: WebSocket, socket_id: str):
    async with SocketService(websocket, socket_id) as connection:
        await connection.listen()


if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=PORT)

FileNotFoundError: [Errno 2] No such file or directory: 'config.json'